<a href="https://colab.research.google.com/github/Maxrgrz/University/blob/main/Extra_tasks/%D0%94%D0%BE%D0%BF%D0%BE%D0%BB%D0%BD%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5%20%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5%20%E2%84%962.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Дополнительное задание №2.2. Замыкания. Декораторы. Итераторы. Генераторы**

**БАЗА:**

- **Замыкания** позволяют создавать функции с сохраняющимся состоянием. Это полезно для создания фабричных функций и функций с настраиваемым поведением.
- **Декораторы** позволяют модифицировать или расширять поведение функций без изменения их исходного кода.

---

## **I. Замыкания и декораторы**

### **Пункт №1**

Напишите две функции создания списка из чётных чисел от 0 до N (N – аргумент функции): \([0, 2, 4, ..., N]\).

- **Первая функция** должна использовать метод `append` для добавления элементов в список.
- **Вторая функция** должна использовать **генератор списков** (list comprehensions) для создания списка.

После этого, через **декоратор**, определите время работы этих функций.

In [10]:
import time

def testTime(fn):

  def wrapper(*args):
    st = time.time()
    fn(*args)
    dt = time.time() - st
    print(f"Время работы: {dt} сек")
    return fn(*args)
  return wrapper


def list_even_nums_1(N: int) -> list:

  lst = []
  for i in range(N):
    if i % 2 == 0:
      lst.append(i)
  return lst

def list_even_nums_2(N: int) -> list:

  return [i for i in range(N) if i%2 == 0]

test1 = testTime(list_even_nums_1)
test2 = testTime(list_even_nums_2)
print(test1(10))
print(test2(10))

Время работы: 5.245208740234375e-06 сек
[0, 2, 4, 6, 8]
Время работы: 5.245208740234375e-06 сек
[0, 2, 4, 6, 8]


---

### **Пункт №2**

Напишите **декоратор** для кэширования результатов работы функции, вычисляющей значение n-го числа [**ряда Фибоначчи**](https://ru.wikipedia.org/wiki/Числа_Фибоначчи).

То есть, при повторном вызове функции через декоратор уже имеющийся результат должен браться из кэша, а не вычисляться заново.

**Например:**

- При значении параметра `n = 5`, должна кэшироваться последовательность \([0, 1, 1, 2, 3, 5]\).
- Вызывая после этого целевую функцию через декоратор ещё раз с `n = 3`, результат \([0, 1, 1, 2]\) должен браться из кэша.
- Если последующее значение `n` больше предыдущего, например `n = 10`, вычисление должно продолжаться, начиная с закэшированной последовательности.

*Подсказка: используйте **замыкание** для хранения кэша внутри декоратора.*


In [15]:
def cache_results(fn):

  cache = [0, 1]
  def wrapper(n):
    if n < len(cache):
      return cache[:n+1]
    for i in range(len(cache), n + 1):
      cache.append(cache[i-1] + cache[i-2])
    return cache
  return wrapper

@cache_results
def fibonacci_count(n:int):

  pass

print(fibonacci_count(5))
print(fibonacci_count(3))
print(fibonacci_count(10))

[0, 1, 1, 2, 3, 5]
[0, 1, 1, 2]
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


---

### **Пункт №3**

Примените к функции из задания №2 сразу **два декоратора**:

1. **Декоратор**, определяющий время выполнения функции.
2. **Кэширующий декоратор** (из задания №2).

Сравните время работы функции с использованием кэширования и без него.


In [17]:
@testTime
@cache_results
def fibonacci_count(n:int):

  pass

print(fibonacci_count(10))
print(fibonacci_count(10))
print(fibonacci_count(100))
print(fibonacci_count(100))

Время работы: 5.4836273193359375e-06 сек
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
Время работы: 2.86102294921875e-06 сек
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
Время работы: 3.4332275390625e-05 сек
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961, 4052739537881, 6557470319842, 10610209857723, 17167680177565, 27777890035288, 44945570212853, 72723460248141, 117669030460994, 190392490709135, 308061521170129, 498454011879264, 806515533049393, 1304969544928657, 2111485077978050, 3416454622906707, 5527939700884757, 89443943

---

### **Пункт №4**

Создайте функцию **make_multiplier(n)**, которая принимает число **n** и возвращает функцию, умножающую переданное ей число на **n**.

**Пример использования:**

```python
def make_multiplier(n):
    # Ваш код

times3 = make_multiplier(3)
print(times3(5))  # Вывод: 15
```

In [ ]:
def make_multiplier(n):
    def multiplier(x):
        return n * x
    return multiplier

times3 = make_multiplier(3)
print(times3(5))

15


---

### **Пункт №5**

Реализуйте функцию с замыканием, которая настраивает округление чисел до заданного количества знаков после запятой.

**Пример:**

```python
def rounder(n):
    # Ваш код

round_to_2 = rounder(2)
print(round_to_2(3.14159))  # Вывод: 3.14
```


In [ ]:
def rounder(n):
    def qez(x):
        return round(x, n)
    return qez

round_to_2 = rounder(2)
print(round_to_2(3.14159))

3.14


---

### **Пункт №6**

Напишите **декоратор**, который измеряет время исполнения функции и выводит его на экран, только если время превышает определённый порог.

**Пример:**

```python
@time_threshold(threshold=0.5)
def long_running_function():
    # Долгий код

long_running_function()
# Выводится время выполнения только если оно больше 0.5 секунд
```

In [18]:
import time
from functools import wraps

def time_threshold(threshold=0.5):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            execution_time = end_time - start_time
            if execution_time > threshold:
                print(f"Функция {func.__name__} выполнена за {execution_time} секунд")
            return result
        return wrapper
    return decorator

@time_threshold(threshold=0.5)
def long_running_function():
    time.sleep(1)  # Имитация долгой функции

long_running_function()

Функция long_running_function выполнена за 1.0010976791381836 секунд


---

## **II. Итераторы и генераторы**

---

### **Пункт №1. Генератор строк фиксированной длины**

Напишите генератор `string_generator(char, times)`, который генерирует строки, состоящие из символа `char`, повторенного от 1 до `times` раз.

```python
# Пример использования:
for s in string_generator('*', 5):
    print(s)
# Вывод:
# *
# **
# ***
# ****
# *****
```



---

In [ ]:
def string_generator(char, times):
    for n in range(1, times + 1):
        yield char * n

for s in string_generator('*', 5):
    print(s)

*
**
***
****
*****


---

### **Пункт №2. Генератор бесконечной последовательности**

Создайте бесконечный генератор `infinite_sequence()`, который с каждым вызовом возвращает следующее число, начиная с 1.

```python
# Пример использования:
gen = infinite_sequence()
for _ in range(5):
    print(next(gen))
# Вывод:
# 1
# 2
# 3
# 4
# 5
```

---

In [ ]:
def infinite_sequence():
    current = 1
    while True:
        yield current
        current += 1

gen = infinite_sequence()
for _ in range(5):
    print(next(gen))

1
2
3
4
5


---

### **Пункт №3. Генератор комбинированных списков**

Создайте генератор `combined_lists(lst1, lst2)`, который попеременно возвращает элементы из `lst1` и `lst2`. Если длины списков неравны, генератор должен остановиться при исчерпании более короткого списка.

```python
# Пример использования:
for item in combined_lists([1, 2, 3], ['a', 'b', 'c', 'd']):
    print(item)
# Вывод:
# 1
# 'a'
# 2
# 'b'
# 3
# 'c'
```

---

In [ ]:
def combined_lists(lst1, lst2):
    k1 = iter(lst1)
    k2 = iter(lst2)
    try:
        while True:
            yield next(k1)
            yield next(k2)
    except StopIteration:
        pass

for item in combined_lists([1, 2, 3], ['a', 'b',"C", 'd']):
    print(item)

1
a
2
b
3
C


---

### **Пункт №4. Генератор перевернутой строки**

Напишите генератор `reverse_string(s)`, который при каждом вызове возвращает следующий символ строки `s` в обратном порядке.

```python
# Пример использования:
for char in reverse_string('hello'):
    print(char)
# Вывод:
# o
# l
# l
# e
# h
```

---

In [ ]:
def reverse_string(s):
  for char in reversed(s):
    yield char

for char in reverse_string('hello'):
    print(char)

o
l
l
e
h


---

### **Пункт №5. Генератор степеней двойки**

Создайте генератор `powers_of_two(n)`, который возвращает степени двойки от 2^0 до 2^n.

```python
# Пример использования:
for num in powers_of_two(5):
    print(num)
# Вывод:
# 1  # 2^0
# 2  # 2^1
# 4  # 2^2
# 8  # 2^3
# 16 # 2^4
# 32 # 2^5
```

---

In [19]:
def powers_of_two(n):
    k = 1
    while n >= 0:
        yield k
        k *= 2
        n -= 1

for num in powers_of_two(5):
    print(num)

1
2
4
8
16
32


---

### **Пункт №6. Генератор чисел из строки**

Напишите генератор `number_extractor(s)`, который извлекает числа из заданной строки `s` и возвращает их как целые числа.

```python
# Пример использования:
for num in number_extractor('abc123def45gh6'):
    print(num)
# Вывод:
# 123
# 45
# 6
```

---

In [ ]:
def number_extractor(string):
  for char in string:
    if char.isdigit() == False:
      string = string.replace(char, ' ')
  listik = string.split(' ')
  while '' in listik:
    listik.remove('')
  for char in range(len(listik)):
    yield listik[char]

for num in number_extractor('abc123def45gh6'):
    print(num)

123
45
6


---